In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, r2_score

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#RM - nombre moyen de pièces par logement
#PTRATIO - ratio élèves-enseignant par ville
#LSTAT -% de statut inférieur de la population
#MEDV - Valeur médiane des logements occupés par leur propriétaire en 1000 $

In [ ]:
df = pd.read_csv('../input/bostonhoustingmlnd/housing.csv')

In [ ]:
df.head(10).T

In [ ]:
df.info()

In [ ]:
df.count()

In [ ]:
#Recherche corrélation

tabcorr = df.corr()  

In [ ]:
tabcorr

In [ ]:
#Carte de température
plt.figure(figsize=(12,12))
sns.heatmap(abs(tabcorr), cmap="coolwarm")

In [ ]:
#Regroupement des paramètres par cluster
sns.clustermap(abs(tabcorr), cmap="coolwarm")

In [ ]:
#dendrogramme des corrélations entre les caractéristiques
from scipy.cluster import hierarchy as hc

corr = 1 - df.corr()
corr_condensed = hc.distance.squareform(corr)
link = hc.linkage(corr_condensed, method='ward')
plt.figure(figsize=(12,12))
den = hc.dendrogram(link, labels=df.columns, orientation='left', leaf_font_size=10)

In [ ]:
#On s'intéresse plus précisément à la valeur des maisons
correlations = tabcorr.MEDV
print(correlations)

In [ ]:
correlations = correlations.drop(['MEDV'],axis=0)

In [ ]:
#Les corrélations fortement négatives sont aussi significatives que les positives ; on considère donc les valeurs absolues, et on trie par ordre décroissant
print(abs(correlations).sort_values(ascending=False))

In [ ]:
#Régression linéaire multiple
df.columns

In [ ]:
continuous_features = ['PTRATIO','LSTAT', 'MEDV']
discrete_features = ['RM']

In [ ]:
df1 = df[df.MEDV<1000000].drop(discrete_features, axis=1)

In [ ]:
X = df1.drop(['MEDV'], axis=1)
y = df1.MEDV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)            # apprentissage
y_pred = lm.predict(X_test)         # prédiction sur l'ensemble de test

In [ ]:
# On trace le nuage de points pour comparer la prédiction et les résultats attendus
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("MEDV")
plt.ylabel("Prediction MEDV")
plt.title("MEDV reels vs predictions")

In [ ]:
#on peut visualiser la distribution de l'erreur avec seaborn
sns.distplot(y_test-y_pred)

In [ ]:
#erreur sur les moindres carrés
print(np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
# rapport des variances estimée/réelle
scoreR2 = r2_score(y_test, y_pred)
print(scoreR2)

In [ ]:
lm.score(X_test,y_test)

In [ ]:
# Régression par forêt aléatoire
X = df.drop(['MEDV'], axis=1)
y = df.MEDV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestRegressor()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)
print(rf.score(X_test,y_test))

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_rf)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("MEDV")
plt.ylabel("Prediction MEDV")
plt.title("MEDV reels vs predictions")

In [ ]:
sns.distplot(y_test-y_rf)

In [ ]:
print(np.sqrt(mean_squared_error(y_test, y_rf)))

In [ ]:
rf.score(X_test,y_test)

In [ ]:
#Importance des caractéristiques
import xgboost as XGB
xgb  = XGB.XGBRegressor()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
print(xgb.score(X_test,y_test))

plt.figure(figsize=(12,12))
plt.scatter(y_test, y_xgb)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("MEDV")
plt.ylabel("Prediction MEDV")
plt.title("MEDV reels vs predictions")